In [1]:
# <your imports>
import numpy as np
import torch
from torch.optim import Adam
import torch.nn as nn
from tqdm import tqdm
from omegaconf import OmegaConf
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import TrainingArguments, Trainer
from transformers import BertForSequenceClassification, XLMRobertaXLConfig
from transformers import EarlyStoppingCallback

from preprocessing import Preprocessing
from matplotlib import pyplot as plt

# models
from models import BertClassifier

# ignore fucking warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
def custom_f1(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    f1_scores = f1_score(y_true=labels, y_pred=pred, average=None)
    final_score = np.mean(f1_scores[1:])
    # return {"custom F1 score": final_score}
    return {
        "F1 for class 0": f1_scores[0],
        "F1 for class 1": f1_scores[1],
        "F1 for class 2": f1_scores[2]
    }

def compute_metrics(p):
    pred, labels = p
    # print(pred, labels)
    # print(np.unique(labels, return_counts=True), labels.shape)
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='weighted')
    precision = precision_score(y_true=labels, y_pred=pred, average='weighted')
    f1 = f1_score(y_true=labels, y_pred=pred, average='weighted')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [3]:
preprocessor = Preprocessing()

# result = preprocessor.get_dataloaders()
result = preprocessor.get_datasets()

train_set = result["train_set"]
val_set = result["val_set"]
# plt.hist(val_set.labels, bins=3)

In [4]:
conf = OmegaConf.load("config.yaml").general
for i in conf.keys():
    print(f'{i}: {conf[i]}')

dataset_name: ruSentNE_lemmatized
batch_size: 64
device: cpu:0
dataloader_shuffle: True
pretrained_model: bert-base-uncased
num_classes: 3
num_workers: 1
lr: 1e-06
epochs: 5
eps: 1e-08
optimizer: AdamW
eval_steps: 10
logging_steps: 10


In [5]:
# help(AutoModelForSequenceClassification)

In [6]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    # "cardiffnlp/twitter-roberta-base-sentiment",
    "coderSounak/finetuned_twitter_sentiment_LSTM", 
    # 'cardiffnlp/twitter-xlm-roberta-base-sentiment',
    num_labels=3,
)

In [7]:
args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=conf.eval_steps,
    per_device_train_batch_size=conf.batch_size,
    per_device_eval_batch_size=conf.batch_size,
    num_train_epochs=conf.epochs,
    seed=42,
    logging_steps=conf.logging_steps,
    # learning_rate=conf.lr,
    learning_rate=5e-7,
    load_best_model_at_end=True,
)

In [8]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_set,
    eval_dataset=val_set,
    # compute_metrics=compute_metrics,
    compute_metrics=custom_f1,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

## Train loop

In [9]:
trainer.train()

Step,Training Loss,Validation Loss,F1 for class 0,F1 for class 1,F1 for class 2
10,1.100400,1.101343,0.518857,0.079051,0.265306
20,1.111000,1.100791,0.510638,0.016667,0.258398
30,1.107100,1.099999,0.514156,0.016807,0.265664
40,1.114500,1.099418,0.513699,0.017094,0.268293
50,1.110600,1.098821,0.515535,0.017094,0.282974
60,1.115000,1.098325,0.501750,0.017241,0.278422
70,1.104100,1.098067,0.504103,0.017316,0.288991
80,1.107600,1.097911,0.501758,0.017391,0.288330
90,1.108400,1.097772,0.501758,0.008734,0.292237
100,1.099200,1.097747,0.502924,0.008734,0.288991


TrainOutput(global_step=240, training_loss=1.1074812332789103, metrics={'train_runtime': 25.675, 'train_samples_per_second': 591.82, 'train_steps_per_second': 9.348, 'total_flos': 53698984128000.0, 'train_loss': 1.1074812332789103, 'epoch': 5.0})

## Evaluation/Weighting metric

In [21]:
import optuna

In [9]:
def make_weighted_metric(p):
    print(p)
    global weights
    pred, labels = p
    if weights is None:
        weights = np.ones(pred.shape[1])
    pred = np.argmax(pred*weights, axis=1)

    f1_scores = f1_score(y_true=labels, y_pred=pred, average=None)
    final_score = np.mean(f1_scores[1:])
    # return {"custom F1 score": final_score}
    return {
        "F1 for class 0": f1_scores[0],
        "F1 for class 1": f1_scores[1],
        "F1 for class 2": f1_scores[2]
    }

In [23]:
def target_func(weight_1, weight_2, weight_3):
    global weights
    weights = np.array([weight_1, weight_2, weight_3])
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_set,
        eval_dataset=val_set,
        # compute_metrics=compute_metrics,
        compute_metrics=make_weighted_metric,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    )
    result = trainer.evaluate()
    final_result = (result['eval_F1 for class 0'] + result['eval_F1 for class 2']) / 2
    return final_result

In [24]:
def objective(trial):
    weight_1 = trial.suggest_float("weight_1", -2, 2)
    weight_2 = trial.suggest_float("weight_2", -2, 2)
    weight_3 = trial.suggest_float("weight_3", -2, 2)
    
    result = target_func(weight_1, weight_2, weight_3)
    
    return result

In [25]:
study = optuna.create_study(directions=["maximize"])
study.optimize(objective, n_trials=30, timeout=300)

[I 2023-05-22 03:48:29,533] A new study created in memory with name: no-name-2fb1bd0d-6119-4a94-9e13-0b4f43885d6a


[I 2023-05-22 03:48:38,311] Trial 0 finished with value: 0.20504201977525868 and parameters: {'weight_1': 1.560514276957245, 'weight_2': -0.42061527092436624, 'weight_3': 0.4874061414526829}. Best is trial 0 with value: 0.20504201977525868.


[I 2023-05-22 03:48:47,084] Trial 1 finished with value: 0.21334341756143527 and parameters: {'weight_1': 0.9872987810231306, 'weight_2': -1.3463606922548554, 'weight_3': 0.682578142523496}. Best is trial 1 with value: 0.21334341756143527.


[I 2023-05-22 03:48:55,898] Trial 2 finished with value: 0.0038610038610038607 and parameters: {'weight_1': 1.3246676104209576, 'weight_2': 0.5358094500508188, 'weight_3': -0.49028174283333525}. Best is trial 1 with value: 0.21334341756143527.


[I 2023-05-22 03:49:04,741] Trial 3 finished with value: 0.11282771535580524 and parameters: {'weight_1': 0.05170723996892024, 'weight_2': -1.2736860389793585, 'weight_3': -1.2602380075915987}. Best is trial 1 with value: 0.21334341756143527.


...

[I 2023-05-22 03:52:21,737] Trial 25 finished with value: 0.2282650215821842 and parameters: {'weight_1': 1.9906145650307194, 'weight_2': -1.6774854135401194, 'weight_3': 1.1048117531000665}. Best is trial 25 with value: 0.2282650215821842.


[I 2023-05-22 03:52:30,729] Trial 26 finished with value: 0.22364072871063406 and parameters: {'weight_1': 1.937541131836139, 'weight_2': -1.9942176913111846, 'weight_3': 1.1790089211647372}. Best is trial 25 with value: 0.2282650215821842.


[I 2023-05-22 03:52:39,716] Trial 27 finished with value: 0.20571888844140154 and parameters: {'weight_1': 1.6548823139576028, 'weight_2': -1.784740711923443, 'weight_3': 1.5090831246772776}. Best is trial 25 with value: 0.2282650215821842.


[I 2023-05-22 03:52:48,712] Trial 28 finished with value: 0.2004914369113589 and parameters: {'weight_1': 1.977153997287923, 'weight_2': -1.964433210353744, 'weight_3': 0.5259350753880117}. Best is trial 25 with value: 0.2282650215821842.


[I 2023-05-22 03:52:57,700] Trial 29 finished with value: 0.20807572635066876 and parameters: {'weight_1': 1.508428238905993, 'weight_2': -1.6876723924401622, 'weight_3': 0.5050462619366317}. Best is trial 25 with value: 0.2282650215821842.


## prediction

In [ ]:
# coming soon